In [3]:
import numpy as np

# Load the data
eeg_data = np.load("muse_eeg_data.npy", allow_pickle=True)
labels = np.load("muse_eeg_labels.npy")

# Print shape of the loaded data
print("EEG Data Shape:", eeg_data.shape)  # (num_samples, time_steps, 4)
print("Labels Shape:", labels.shape)  # (num_samples,)

# Print first EEG sample (raw data from 4 channels)
print("First EEG Sample:\n", eeg_data[49])
print(eeg_data.shape[0])
# Print first label
print("First Label:", labels[0])

EEG Data Shape: (50,)
Labels Shape: (50,)
First EEG Sample:
 [[   0.          -27.34375     -80.078125   -109.86328125]
 [ -57.12890625  -20.01953125   -5.37109375 -172.36328125]
 [ -71.2890625   -65.91796875 -193.359375   -177.734375  ]
 ...
 [-141.6015625   -26.85546875  528.80859375   -9.27734375]
 [-141.6015625   -72.265625    414.55078125  -20.5078125 ]
 [-143.06640625 -147.4609375   269.53125     -33.69140625]]
50
First Label: 0


In [4]:
flattened_samples = [np.concatenate(sample).astype(np.float32) for sample in eeg_data]
flattened_samples[0]

array([-47.851562, -40.039062, -57.617188, ..., -10.253906, -18.066406,
       -93.75    ], dtype=float32)

In [5]:
fixed_length = max(len(sample) for sample in flattened_samples)

# Function to pad or truncate
def pad_or_truncate(sample, length):
    if len(sample) < length:
        return np.pad(sample, (0, length - len(sample)), mode="constant")
    else:
        return sample[:length]

# Apply padding/truncation
eeg_features = np.array([pad_or_truncate(sample, fixed_length) for sample in flattened_samples], dtype=np.float32)

print("Fixed EEG Features Shape:", eeg_features.shape)

Fixed EEG Features Shape: (50, 10272)


In [6]:
import faiss

embedding_dim = eeg_features.shape[1]  # Feature dimension

# Create FAISS index
index = faiss.IndexFlatL2(embedding_dim)
index.add(eeg_features)

print("FAISS index successfully created with", index.ntotal, "entries!")


FAISS index successfully created with 50 entries!


In [7]:
def find_closest_label(query_sample, top_k=1):
    """Find the closest EEG vector in FAISS and return its label."""
    query_sample = np.array(query_sample, dtype=np.float32).reshape(1, -1)  # Ensure correct shape
    distances, indices = index.search(query_sample, top_k)  # Find closest match
    
    closest_labels = [labels[i] for i in indices[0]]  # Get corresponding labels
    return closest_labels, distances

# Example Query (Use any sample from EEG dataset)
query_sample = eeg_data[12]  # Replace with real input
print((query_sample.shape))
closest_labels, distances = find_closest_label(query_sample)

print("Closest Label(s):", closest_labels)
print("Distances:", distances)


(2568, 4)
Closest Label(s): [2]
Distances: [[0.]]


In [14]:
# Load EEG data
filename = "eeg_recording.txt"
eeg_data = np.loadtxt(filename, dtype=np.float32)

# Define target shape
target_samples = 2568
num_channels = 4

# Ensure it's a NumPy array
eeg_data = np.array(eeg_data, dtype=np.float32)

# Handle cases where data is shorter or longer
if eeg_data.shape[0] < target_samples:
    # Pad with zeros if too short
    padding = np.zeros((target_samples - eeg_data.shape[0], num_channels), dtype=np.float32)
    eeg_data = np.vstack((eeg_data, padding))
elif eeg_data.shape[0] > target_samples:
    # Truncate if too long
    eeg_data = eeg_data[:target_samples, :]

# Check final shape
print("Final EEG Shape:", eeg_data.shape)  # Should be (2568, 4)


Final EEG Shape: (2568, 4)


In [15]:
THOUGHT_LABELS = ["dhp", "sab", "fbh", "cfm", "null"]  # Thought categories


In [16]:
closest_labels, distances = find_closest_label(eeg_data)

print("Closest Label(s):", THOUGHT_LABELS[closest_labels[0]])
print("Distances:", distances)

Closest Label(s): cfm
Distances: [[4.9611357e+08]]
